In [1]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 4
image_size = 224
batch_size_training = 10
batch_size_validation = 10


In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'New folder/train',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

validation_generator = data_generator.flow_from_directory(
    'New folder/validation',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

Found 232 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [5]:
#defingin the model
model = Sequential()

In [6]:
#adding VGG16 model
from xml.etree.ElementInclude import include


model.add(
    ResNet50(include_top = False, pooling='avg', weights ='imagenet')
    )

94765736/94765736 [==============================] - 155s 2us/step


In [7]:
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(num_classes, activation='softmax'))
model.layers

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dense_1 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 31,996,804
Trainable params: 31,943,684
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
#number of layers
model.layers[1]

In [10]:
#training the model
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])
steps_per_epoch_train = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
number_epochs = 2

In [11]:
steps_per_epoch_train

24

In [12]:
steps_per_epoch_validation

1

In [13]:
for layer in model.layers[0].layers:

    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000002991947E400> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000299194D22E0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x00000299194D2580> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000299194D2A30> False
<keras.layers.core.activation.Activation object at 0x00000299194D25B0> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x000002991967B580> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x00000299196CEA90> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000002991972D8E0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000029919737B50> False
<keras.layers.core.activation.Activation object at 0x000002991972DDC0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000029919743370> False
<keras.layers.normalization.batch_normalization.BatchNormalization obj

In [14]:
fit_history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=number_epochs,verbose=1, validation_data=validation_generator, validation_steps=steps_per_epoch_validation,)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20384\2980021967.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=number_epochs,verbose=1, validation_data=validation_generator, validation_steps=steps_per_epoch_validation,)


Epoch 1/2
24/24 [==============================] - 22s 765ms/step - loss: 0.9128 - accuracy: 0.8362 - val_loss: 0.8928 - val_accuracy: 0.8889
Epoch 2/2
24/24 [==============================] - 17s 709ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.4226 - val_accuracy: 0.8889


In [16]:
model.save('model/ResNet50/ResNet50_Product_Classification_2Dense.h5')

In [14]:
#save the model
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [1]:
from keras.models import load_model
ResNet50_saved = load_model('model/ResNet50/ResNet50_Product_Classification_2Dense.h5')

In [18]:
ResNet50_saved.layers[1]

In [19]:
ResNet50_saved.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dense_1 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 31,996,804
Trainable params: 8,409,092
Non-trainable params: 23,587,712
_________________________________________________________________


In [2]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image = load_img('New folder/test/test.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

# predict the probability across all output classes
yhat = ResNet50_saved.predict(image)




1/1 [==============================] - 1s 941ms/step


In [3]:
yhat[0]

array([9.9999976e-01, 8.0185835e-08, 3.9740991e-16, 1.4666466e-07],
      dtype=float32)

In [20]:
import numpy as np

In [4]:
from keras.models import Model
basemodel = Model(inputs=ResNet50_saved.input, outputs=ResNet50_saved.get_layer('dense').output)

In [18]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

1/1 [==============================] - 0s 146ms/step


In [31]:
yhat1_base

array([[0.9822604, 2.2466671, 0.       , ..., 0.       , 0.       ,
        0.6798813]], dtype=float32)

In [19]:
# import required libraries
import numpy as np
from numpy.linalg import norm

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

#VGG16 - 0.9403707
#ResNet50 - 0.9701082

Cosine Similarity: 0.9701082


In [7]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test2.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

1/1 [==============================] - 0s 124ms/step


In [8]:
# import required libraries
import numpy as np
from numpy.linalg import norm

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

#VGG16 - 0.47206444
#ResNet50 - 0.40083203

Cosine Similarity: 0.40083203


In [9]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 140ms/step
Cosine Similarity: 0.8852835


In [10]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test5.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 147ms/step
Cosine Similarity: 0.56222504


In [12]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test2.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test5.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 133ms/step
Cosine Similarity: 0.76314044


In [13]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test6.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test5.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 130ms/step
Cosine Similarity: 0.92447865


In [14]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test7.jpeg', target_size=(224, 224))
image2 = load_img('New folder/test/test5.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 134ms/step
Cosine Similarity: 0.9828984


In [15]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test7.jpeg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 140ms/step
Cosine Similarity: 0.5496688


In [17]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test5.png', target_size=(224, 224))
image2 = load_img('New folder/test/test4.jpg', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)
#0.56

1/1 [==============================] - 0s 129ms/step
Cosine Similarity: 0.6096772
